In [1]:
import sqlite3
import time
import queries


In [2]:
queryResponseTimes = {}
Queries = queries.returnQueries2()
print(len(Queries))

4


In [4]:
for i in range(11):
    print(f'Starting Group {i}')
    for q in Queries:
        if q != 'q17' and q != 'q22':
            query = Queries[q]
            conn = sqlite3.connect('tpch.db')
            cursor = conn.cursor()
            start_time = time.time()  # Capture the start time
            cursor.execute(query)
            results = cursor.fetchall()
            # print(results)
            end_time = time.time()
            cursor.close()
            conn.close()
            duration = end_time - start_time
            print(f"{q} took {duration} seconds to execute.")
            if q in queryResponseTimes:
                queryResponseTimes[q].append(duration)
            else:
                queryResponseTimes[q] = [duration]
    print('-'*30)



Starting Group 0
q17_drop_view took 0.001577138900756836 seconds to execute.
q17_create_view took 0.0010371208190917969 seconds to execute.
q17_call took 25.248572826385498 seconds to execute.
q22_v3 took 1.8942530155181885 seconds to execute.
------------------------------
Starting Group 1
q17_drop_view took 0.00144195556640625 seconds to execute.
q17_create_view took 0.0010938644409179688 seconds to execute.
q17_call took 25.10322380065918 seconds to execute.
q22_v3 took 1.9119789600372314 seconds to execute.
------------------------------
Starting Group 2
q17_drop_view took 0.0015590190887451172 seconds to execute.
q17_create_view took 0.0009999275207519531 seconds to execute.
q17_call took 25.01981806755066 seconds to execute.
q22_v3 took 1.9175050258636475 seconds to execute.
------------------------------
Starting Group 3
q17_drop_view took 0.0015261173248291016 seconds to execute.
q17_create_view took 0.0012941360473632812 seconds to execute.
q17_call took 24.554089069366455 sec

In [5]:
# print(queryResponseTimes)
import pandas as pd
df = pd.DataFrame(queryResponseTimes)
df.to_csv('17and22.csv')


In [7]:
query = '''select
sum(l_extendedprice) / 7.0 as avg_yearly
from
lineitem,
part
where
p_partkey = l_partkey
and p_brand = 'Brand#52'
and p_container = 'JUMBO CAN'
and l_quantity < (
select
0.2 * avg(l_quantity)
from
lineitem
where
l_partkey = p_partkey
);'''

conn = sqlite3.connect('tpch.db')
cursor = conn.cursor()
start_time = time.time()  # Capture the start time
cursor.execute(query)
results = cursor.fetchall()
end_time = time.time()
cursor.close()
conn.close()
duration = end_time - start_time
print(f"Query took {duration} seconds to execute.")

In [7]:
query = '''WITH AvgQuantities AS (
    SELECT 
        l_partkey, 0.2 * AVG(l_quantity) AS avg_quantity
    FROM
        lineitem
    GROUP BY
        l_partkey
)
SELECT
    SUM(l.l_extendedprice) / 7.0 AS avg_yearly
FROM
    part p
JOIN
    lineitem l ON p.p_partkey = l.l_partkey
JOIN
    AvgQuantities aq ON l.l_partkey = aq.l_partkey
WHERE
    p.p_brand = 'Brand#52'
    AND p.p_container = 'JUMBO CAN'
    AND l.l_quantity < aq.avg_quantity;

'''

conn = sqlite3.connect('tpch.db')
cursor = conn.cursor()
start_time = time.time()  # Capture the start time
cursor.execute(query)
results = cursor.fetchall()
end_time = time.time()
cursor.close()
conn.close()
duration = end_time - start_time
print(f"Query took {duration} seconds to execute.")


Query took 12.21946096420288 seconds to execute.
